<a href="https://colab.research.google.com/github/younesabdolmalaky/Using-image-processing-techniques-for-audio-classification/blob/main/classification_spectogram_6_dB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/sp/Spectrogram1.zip -d /content/Spectrogram
!unzip /content/drive/MyDrive/sp/Spectrogram2.zip -d /content/Spectrogram
!unzip /content/drive/MyDrive/sp/Spectrogram3.zip -d /content/Spectrogram

Streaming output truncated to the last 5000 lines.
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9231.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9253.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9957.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/8935.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9526.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/8943.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9866.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9368.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_02/normal/9743.png  
  inflating: /content/Spectrogram/content/Spectrogram/0_dB_slider/slider/id_

move all folders to Spectrogram

In [4]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
import glob
from tensorflow import keras
from keras.layers.core import Dense 
from tensorflow.keras.layers import Input
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras import callbacks
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

%matplotlib inline

In [5]:
model_conv = InceptionV3(weights='imagenet', input_shape=(150, 150, 3), include_top=False)


87910968/87910968 [==============================] - 0s 0us/step


In [6]:
model_conv.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [7]:
def create_model():
  model_conv.trainable = True
  model = models.Sequential()
  model.add(model_conv)
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'binary_accuracy'])
  return model

  
  

In [8]:
def load_data(path): 
  batch_size = 128
  img_height = 150 
  img_width = 150
  data_gen = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2) # set validation split

  train_generator = data_gen.flow_from_directory(
      path,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='binary',
      subset='training') # set as training data

  validation_generator = data_gen.flow_from_directory(
      path, # same directory as training data
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='binary',
      subset='validation') # set as validation data
  return train_generator ,  validation_generator


In [9]:
def fit(path , epoch):
  model = create_model()
  train_generator , validation_generator = load_data(path)
  mch = callbacks.ModelCheckpoint(path+'/fianl-model.h5' , monitor='val_loss' , mode ='min' , save_best_only=True)
  class_weights = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(train_generator.labels),y = train_generator.labels)
  class_weights = {i : class_weights[i] for i in range(2)}
  model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=epoch, validation_data=validation_generator, validation_steps=len(validation_generator) , callbacks=[mch] , class_weight = class_weights)



In [10]:
epoch = 200

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_00'
fit(path,epoch)

Found 1135 images belonging to 2 classes.
Found 283 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 39s 2s/step - loss: 1.8402 - precision: 0.7230 - recall: 0.6613 - binary_accuracy: 0.5780 - val_loss: 1.9574 - val_precision: 0.7138 - val_recall: 1.0000 - val_binary_accuracy: 0.7138
Epoch 2/200
9/9 [==============================] - 8s 852ms/step - loss: 0.6968 - precision: 0.7174 - recall: 0.2855 - binary_accuracy: 0.4106 - val_loss: 2.5925 - val_precision: 0.7138 - val_recall: 1.0000 - val_binary_accuracy: 0.7138
Epoch 3/200
9/9 [==============================] - 7s 779ms/step - loss: 0.7216 - precision: 0.7972 - recall: 0.3498 - binary_accuracy: 0.4731 - val_loss: 15.7070 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_binary_accuracy: 0.2862
Epoch 4/200
9/9 [==============================] - 10s 1s/step - loss: 0.6713 - precision: 0.7948 - recall: 0.4549 - binary_accuracy: 0.5278 - val_loss: 1.0376 - val_precision: 0.0000e+00 - val_recal

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_02'
fit(path,epoch)

Found 1101 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 21s 2s/step - loss: 2.0556 - precision_1: 0.7545 - recall_1: 0.5178 - binary_accuracy: 0.5195 - val_loss: 243.0844 - val_precision_1: 0.6667 - val_recall_1: 0.0197 - val_binary_accuracy: 0.2664
Epoch 2/200
9/9 [==============================] - 8s 846ms/step - loss: 1.4475 - precision_1: 0.7627 - recall_1: 0.6999 - binary_accuracy: 0.6176 - val_loss: 51998.3594 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_binary_accuracy: 0.2591
Epoch 3/200
9/9 [==============================] - 7s 817ms/step - loss: 1.5118 - precision_1: 0.7813 - recall_1: 0.7601 - binary_accuracy: 0.6658 - val_loss: 22112.5156 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_binary_accuracy: 0.2591
Epoch 4/200
9/9 [==============================] - 7s 747ms/step - loss: 1.2179 - precision_1: 0.7658 - recall_1: 0.8044 - binary_accuracy: 0.6739 - val_loss:

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_04'
fit(path,epoch)

Found 1106 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 24s 2s/step - loss: 0.9650 - precision_2: 0.7924 - recall_2: 0.7291 - binary_accuracy: 0.6546 - val_loss: 3.9362 - val_precision_2: 0.7816 - val_recall_2: 0.3301 - val_binary_accuracy: 0.4291
Epoch 2/200
9/9 [==============================] - 8s 906ms/step - loss: 0.6488 - precision_2: 0.8376 - recall_2: 0.8356 - binary_accuracy: 0.7559 - val_loss: 4.5045 - val_precision_2: 0.8138 - val_recall_2: 0.9757 - val_binary_accuracy: 0.8145
Epoch 3/200
9/9 [==============================] - 8s 953ms/step - loss: 0.4011 - precision_2: 0.9172 - recall_2: 0.9105 - binary_accuracy: 0.8716 - val_loss: 0.4071 - val_precision_2: 0.8640 - val_recall_2: 0.9563 - val_binary_accuracy: 0.8545
Epoch 4/200
9/9 [==============================] - 8s 821ms/step - loss: 0.1813 - precision_2: 0.9739 - recall_2: 0.9480 - binary_accuracy: 0.9421 - val_loss: 0.5498 - val_precision_2:

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_fan/fan/id_06'
fit(path,epoch)

Found 1101 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 36s 2s/step - loss: 3.3030 - precision: 0.8192 - recall: 0.6084 - binary_accuracy: 0.6122 - val_loss: 20.8425 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_binary_accuracy: 0.2618
Epoch 2/200
9/9 [==============================] - 9s 947ms/step - loss: 0.3830 - precision: 0.9532 - recall: 0.9039 - binary_accuracy: 0.8965 - val_loss: 11.6458 - val_precision: 0.7382 - val_recall: 1.0000 - val_binary_accuracy: 0.7382
Epoch 3/200
9/9 [==============================] - 7s 721ms/step - loss: 0.5866 - precision: 0.9242 - recall: 0.7512 - binary_accuracy: 0.7711 - val_loss: 2581.1121 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_binary_accuracy: 0.2618
Epoch 4/200
9/9 [==============================] - 7s 754ms/step - loss: 0.4036 - precision: 0.9410 - recall: 0.9618 - binary_accuracy: 0.9273 - val_loss: 1558.7162 - val_precision: 0.000

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_00'
fit(path,epoch)

Found 920 images belonging to 2 classes.
Found 229 images belonging to 2 classes.
Epoch 1/200
8/8 [==============================] - 22s 2s/step - loss: 1.8514 - precision_1: 0.9060 - recall_1: 0.6348 - binary_accuracy: 0.6228 - val_loss: 1633.0419 - val_precision_1: 0.8727 - val_recall_1: 0.9552 - val_binary_accuracy: 0.8384
Epoch 2/200
8/8 [==============================] - 8s 1s/step - loss: 0.9741 - precision_1: 0.8860 - recall_1: 0.3379 - binary_accuracy: 0.3826 - val_loss: 57.4509 - val_precision_1: 0.8777 - val_recall_1: 1.0000 - val_binary_accuracy: 0.8777
Epoch 3/200
8/8 [==============================] - 8s 1s/step - loss: 0.6488 - precision_1: 0.9006 - recall_1: 0.9006 - binary_accuracy: 0.8261 - val_loss: 1.5859 - val_precision_1: 0.8555 - val_recall_1: 0.7363 - val_binary_accuracy: 0.6594
Epoch 4/200
8/8 [==============================] - 6s 678ms/step - loss: 2.3477 - precision_1: 0.9501 - recall_1: 0.6621 - binary_accuracy: 0.6739 - val_loss: 17.8838 - val_precision_1: 0

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_02'
fit(path,epoch)

Found 893 images belonging to 2 classes.
Found 223 images belonging to 2 classes.
Epoch 1/200
7/7 [==============================] - 24s 2s/step - loss: 0.6750 - precision_2: 0.9429 - recall_2: 0.8831 - binary_accuracy: 0.8466 - val_loss: 35.7895 - val_precision_2: 0.5556 - val_recall_2: 0.0249 - val_binary_accuracy: 0.1031
Epoch 2/200
7/7 [==============================] - 7s 951ms/step - loss: 0.7407 - precision_2: 0.9775 - recall_2: 0.9179 - binary_accuracy: 0.9071 - val_loss: 1905.1021 - val_precision_2: 0.9388 - val_recall_2: 0.2289 - val_binary_accuracy: 0.2915
Epoch 3/200
7/7 [==============================] - 6s 782ms/step - loss: 0.1715 - precision_2: 0.9884 - recall_2: 0.9565 - binary_accuracy: 0.9507 - val_loss: 11841.5586 - val_precision_2: 0.9677 - val_recall_2: 0.2985 - val_binary_accuracy: 0.3587
Epoch 4/200
7/7 [==============================] - 6s 767ms/step - loss: 0.1213 - precision_2: 0.9936 - recall_2: 0.9726 - binary_accuracy: 0.9698 - val_loss: 14625.3770 - val_p

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_04'
fit(path,epoch)

Found 642 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Epoch 1/200
6/6 [==============================] - 18s 2s/step - loss: 0.7357 - precision_3: 0.8734 - recall_3: 0.7242 - binary_accuracy: 0.6667 - val_loss: 157.6059 - val_precision_3: 0.8750 - val_recall_3: 1.0000 - val_binary_accuracy: 0.8750
Epoch 2/200
6/6 [==============================] - 6s 888ms/step - loss: 0.6693 - precision_3: 0.8997 - recall_3: 0.6388 - binary_accuracy: 0.6215 - val_loss: 2546.8960 - val_precision_3: 0.8750 - val_recall_3: 1.0000 - val_binary_accuracy: 0.8750
Epoch 3/200
6/6 [==============================] - 4s 615ms/step - loss: 0.9599 - precision_3: 0.9576 - recall_3: 0.2011 - binary_accuracy: 0.2928 - val_loss: 4154.6895 - val_precision_3: 0.8750 - val_recall_3: 1.0000 - val_binary_accuracy: 0.8750
Epoch 4/200
6/6 [==============================] - 4s 777ms/step - loss: 2.8286 - precision_3: 0.9531 - recall_3: 0.1085 - binary_accuracy: 0.2150 - val_loss: 6914.5469 - val_pr

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_pump/pump/id_06'
fit(path,epoch)

Found 911 images belonging to 2 classes.
Found 227 images belonging to 2 classes.
Epoch 1/200
8/8 [==============================] - 23s 2s/step - loss: 0.6530 - precision_4: 0.9615 - recall_4: 0.3016 - binary_accuracy: 0.3535 - val_loss: 1.4559 - val_precision_4: 0.8793 - val_recall_4: 0.4928 - val_binary_accuracy: 0.4758
Epoch 2/200
8/8 [==============================] - 7s 890ms/step - loss: 0.6676 - precision_4: 0.9625 - recall_4: 0.3100 - binary_accuracy: 0.3611 - val_loss: 0.7125 - val_precision_4: 0.8636 - val_recall_4: 0.2754 - val_binary_accuracy: 0.2996
Epoch 3/200
8/8 [==============================] - 7s 762ms/step - loss: 0.6539 - precision_4: 0.9447 - recall_4: 0.2678 - binary_accuracy: 0.3194 - val_loss: 0.8892 - val_precision_4: 0.8594 - val_recall_4: 0.2657 - val_binary_accuracy: 0.2907
Epoch 4/200
8/8 [==============================] - 7s 808ms/step - loss: 1.4795 - precision_4: 0.9845 - recall_4: 0.3064 - binary_accuracy: 0.3644 - val_loss: 8.4005 - val_precision_4: 

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/slider/id_00'
fit(path,epoch)

Found 1140 images belonging to 2 classes.
Found 284 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 27s 2s/step - loss: 1.2521 - precision_6: 0.7548 - recall_6: 0.1836 - binary_accuracy: 0.3430 - val_loss: 0.7125 - val_precision_6: 1.0000 - val_recall_6: 0.0141 - val_binary_accuracy: 0.2606
Epoch 2/200
9/9 [==============================] - 10s 1s/step - loss: 0.7547 - precision_6: 0.7576 - recall_6: 0.3509 - binary_accuracy: 0.4289 - val_loss: 0.6909 - val_precision_6: 0.7500 - val_recall_6: 1.0000 - val_binary_accuracy: 0.7500
Epoch 3/200
9/9 [==============================] - 8s 896ms/step - loss: 0.7690 - precision_6: 0.7704 - recall_6: 0.4865 - binary_accuracy: 0.5061 - val_loss: 0.6945 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_binary_accuracy: 0.2500
Epoch 4/200
9/9 [==============================] - 9s 944ms/step - loss: 0.6988 - precision_6: 0.8194 - recall_6: 0.7427 - binary_accuracy: 0.6842 - val_loss: 0.6553 - val_precis

In [ ]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/slider/id_02'
fit(path,epoch)

Found 1069 images belonging to 2 classes.
Found 266 images belonging to 2 classes.
Epoch 1/200
9/9 [==============================] - 20s 1s/step - loss: 0.5035 - precision_7: 0.9085 - recall_7: 0.9287 - binary_accuracy: 0.8681 - val_loss: 0.3124 - val_precision_7: 0.9119 - val_recall_7: 0.9718 - val_binary_accuracy: 0.9023
Epoch 2/200
9/9 [==============================] - 8s 889ms/step - loss: 0.4401 - precision_7: 0.9135 - recall_7: 0.9637 - binary_accuracy: 0.8980 - val_loss: 0.3105 - val_precision_7: 0.9134 - val_recall_7: 0.9906 - val_binary_accuracy: 0.9173
Epoch 3/200
9/9 [==============================] - 8s 969ms/step - loss: 0.4097 - precision_7: 0.9157 - recall_7: 0.9906 - binary_accuracy: 0.9196 - val_loss: 0.2580 - val_precision_7: 0.9254 - val_recall_7: 0.9906 - val_binary_accuracy: 0.9286
Epoch 4/200
9/9 [==============================] - 7s 797ms/step - loss: 0.3983 - precision_7: 0.9208 - recall_7: 0.9930 - binary_accuracy: 0.9261 - val_loss: 0.2942 - val_precision_7:

In [12]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/slider/id_04'
fit(path,epoch)

Found 571 images belonging to 2 classes.
Found 141 images belonging to 2 classes.
Epoch 1/200
5/5 [==============================] - 59s 3s/step - loss: 4.9560 - precision_1: 0.7404 - recall_1: 0.4065 - binary_accuracy: 0.4483 - val_loss: 5.4988 - val_precision_1: 0.7518 - val_recall_1: 1.0000 - val_binary_accuracy: 0.7518
Epoch 2/200
5/5 [==============================] - 5s 905ms/step - loss: 0.9642 - precision_1: 0.7611 - recall_1: 0.7220 - binary_accuracy: 0.6217 - val_loss: 665.1154 - val_precision_1: 0.7518 - val_recall_1: 1.0000 - val_binary_accuracy: 0.7518
Epoch 3/200
5/5 [==============================] - 4s 866ms/step - loss: 0.7120 - precision_1: 0.7871 - recall_1: 0.5701 - binary_accuracy: 0.5622 - val_loss: 2179.1431 - val_precision_1: 0.7518 - val_recall_1: 1.0000 - val_binary_accuracy: 0.7518
Epoch 4/200
5/5 [==============================] - 4s 719ms/step - loss: 0.8300 - precision_1: 0.8036 - recall_1: 0.5257 - binary_accuracy: 0.5482 - val_loss: 2486.6746 - val_preci

In [13]:
path = '/content/Spectrogram/content/Spectrogram/-6_dB_slider/slider/id_06'
fit(path,epoch)

Found 500 images belonging to 2 classes.
Found 123 images belonging to 2 classes.
Epoch 1/200
4/4 [==============================] - 46s 4s/step - loss: 0.8449 - precision_2: 0.9016 - recall_2: 0.6425 - binary_accuracy: 0.6340 - val_loss: 0.1952 - val_precision_2: 0.8618 - val_recall_2: 1.0000 - val_binary_accuracy: 0.8618
Epoch 2/200
4/4 [==============================] - 3s 807ms/step - loss: 0.1703 - precision_2: 0.9881 - recall_2: 0.9696 - binary_accuracy: 0.9640 - val_loss: 0.2909 - val_precision_2: 0.8983 - val_recall_2: 1.0000 - val_binary_accuracy: 0.9024
Epoch 3/200
4/4 [==============================] - 6s 2s/step - loss: 0.1079 - precision_2: 0.9928 - recall_2: 0.9720 - binary_accuracy: 0.9700 - val_loss: 0.0034 - val_precision_2: 1.0000 - val_recall_2: 1.0000 - val_binary_accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 3s 805ms/step - loss: 0.1682 - precision_2: 1.0000 - recall_2: 0.9159 - binary_accuracy: 0.9280 - val_loss: 167.2808 - val_precision_2: 1

In [14]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_00'
fit(path,epoch)

Found 889 images belonging to 2 classes.
Found 221 images belonging to 2 classes.
Epoch 1/200
7/7 [==============================] - 49s 2s/step - loss: 0.7680 - precision_3: 0.9490 - recall_3: 0.7982 - binary_accuracy: 0.7818 - val_loss: 2717.5488 - val_precision_3: 0.9239 - val_recall_3: 0.8586 - val_binary_accuracy: 0.8100
Epoch 2/200
7/7 [==============================] - 7s 1s/step - loss: 0.4072 - precision_3: 0.9851 - recall_3: 0.7491 - binary_accuracy: 0.7660 - val_loss: 877.8637 - val_precision_3: 0.8959 - val_recall_3: 1.0000 - val_binary_accuracy: 0.8959
Epoch 3/200
7/7 [==============================] - 7s 900ms/step - loss: 0.2439 - precision_3: 0.9955 - recall_3: 0.8449 - binary_accuracy: 0.8583 - val_loss: 3044.7559 - val_precision_3: 0.8959 - val_recall_3: 1.0000 - val_binary_accuracy: 0.8959
Epoch 4/200
7/7 [==============================] - 7s 1s/step - loss: 0.0636 - precision_3: 1.0000 - recall_3: 0.9660 - binary_accuracy: 0.9696 - val_loss: 711.5563 - val_precision

In [15]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_02'
fit(path,epoch)

Found 663 images belonging to 2 classes.
Found 165 images belonging to 2 classes.
Epoch 1/200
6/6 [==============================] - 45s 2s/step - loss: 0.9401 - precision_4: 0.9053 - recall_4: 0.4215 - binary_accuracy: 0.4676 - val_loss: 1.0481 - val_precision_4: 0.8545 - val_recall_4: 1.0000 - val_binary_accuracy: 0.8545
Epoch 2/200
6/6 [==============================] - 4s 658ms/step - loss: 0.5083 - precision_4: 0.9215 - recall_4: 0.9524 - binary_accuracy: 0.8899 - val_loss: 2.2695 - val_precision_4: 0.8537 - val_recall_4: 0.9929 - val_binary_accuracy: 0.8485
Epoch 3/200
6/6 [==============================] - 5s 915ms/step - loss: 1.9598 - precision_4: 0.8959 - recall_4: 0.9259 - binary_accuracy: 0.8446 - val_loss: 539.8720 - val_precision_4: 0.8545 - val_recall_4: 1.0000 - val_binary_accuracy: 0.8545
Epoch 4/200
6/6 [==============================] - 5s 761ms/step - loss: 0.3085 - precision_4: 0.9898 - recall_4: 0.8519 - binary_accuracy: 0.8658 - val_loss: 5629.3110 - val_precisio

In [16]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_04'
fit(path,epoch)

Found 896 images belonging to 2 classes.
Found 224 images belonging to 2 classes.
Epoch 1/200
7/7 [==============================] - 44s 1s/step - loss: 0.7589 - precision_5: 0.9173 - recall_5: 0.8037 - binary_accuracy: 0.7600 - val_loss: 3.4541 - val_precision_5: 0.9171 - val_recall_5: 0.8850 - val_binary_accuracy: 0.8259
Epoch 2/200
7/7 [==============================] - 7s 970ms/step - loss: 0.7974 - precision_5: 0.9871 - recall_5: 0.7638 - binary_accuracy: 0.7801 - val_loss: 449.8479 - val_precision_5: 0.9091 - val_recall_5: 0.1500 - val_binary_accuracy: 0.2277
Epoch 3/200
7/7 [==============================] - 7s 1s/step - loss: 0.1770 - precision_5: 0.9945 - recall_5: 0.9062 - binary_accuracy: 0.9118 - val_loss: 38.4088 - val_precision_5: 0.9718 - val_recall_5: 0.6900 - val_binary_accuracy: 0.7054
Epoch 4/200
7/7 [==============================] - 6s 774ms/step - loss: 0.1542 - precision_5: 0.9973 - recall_5: 0.9362 - binary_accuracy: 0.9408 - val_loss: 41.2781 - val_precision_5:

In [17]:
path ='/content/Spectrogram/content/Spectrogram/-6_dB_valve/valve/id_06'
fit(path,epoch)

Found 890 images belonging to 2 classes.
Found 222 images belonging to 2 classes.
Epoch 1/200
7/7 [==============================] - 47s 2s/step - loss: 0.8481 - precision_6: 0.9451 - recall_6: 0.6725 - binary_accuracy: 0.6730 - val_loss: 1.5906 - val_precision_6: 0.9431 - val_recall_6: 0.5859 - val_binary_accuracy: 0.5991
Epoch 2/200
7/7 [==============================] - 9s 1s/step - loss: 0.5260 - precision_6: 0.9721 - recall_6: 0.6587 - binary_accuracy: 0.6787 - val_loss: 1.2610 - val_precision_6: 0.9394 - val_recall_6: 0.7828 - val_binary_accuracy: 0.7613
Epoch 3/200
7/7 [==============================] - 7s 988ms/step - loss: 0.2799 - precision_6: 0.9855 - recall_6: 0.8577 - binary_accuracy: 0.8618 - val_loss: 0.6260 - val_precision_6: 0.9880 - val_recall_6: 0.8333 - val_binary_accuracy: 0.8423
Epoch 4/200
7/7 [==============================] - 8s 1s/step - loss: 0.1661 - precision_6: 0.9885 - recall_6: 0.9736 - binary_accuracy: 0.9663 - val_loss: 0.4155 - val_precision_6: 0.9943